In [16]:
import boto3
import pandas as pd
from datetime import datetime
import re

s3 = boto3.client('s3')

In [34]:
def get_all_s3_keys(bucket):
    """Get a list of all keys in an S3 bucket."""
    keysPlayers = []
    keysTournaments = []

    kwargs = {'Bucket': bucket}
    while True:
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            if obj['Key'].split('/')[0] == 'tennis' and obj['Key'].split('/')[1] == "Players" and int(obj['Key'].split('/')[2]) == 2020:
                keysPlayers.append(obj['Key'])
            if obj['Key'].split('/')[0] == 'tennis' and obj['Key'].split('/')[1] == "Tournaments" and int(obj['Key'].split('/')[2]) == 2020:
                keysTournaments.append(obj['Key'])
                
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

    return keysPlayers, keysTournaments

keysPlayers, keysTournaments = get_all_s3_keys("pronoo")

In [35]:
all_tournaments = pd.DataFrame([], columns=['name', 'phase', 'surfaceInout', 'typeTournament', 'date', 'winnerRate',
       'looserRate', 'looserName', 'winnerName', 'winnerRateSurface',
       'looserRateSurface', 'url'])

all_players = pd.DataFrame([], columns=['player', 'aces', 'doubleFautes', 'firstServe', 'firstServePointsWon',
       'secondServePointsWon', 'breakPointsFaced', 'breakPointsSaved',
       'serviceGamePlayed', 'serviceGamesWon', 'totalServicePointsWon',
       'firstServeReturnPointsWon', 'secondServeReturnPointsWon',
       'breakPointsOpportunities', 'breakPointsConverted', 'returnGamesPlayed',
       'returnGamesWon', 'returnPointsWon', 'totalPointsWon', 'url', 'year',
       'weight', 'height', 'age', 'startCareer', 'plays', 'ranking', 'points',
       'tournCell', 'pointsDropping', 'nextBest'])

for i,key in enumerate(keysPlayers):
    if i%5000==0:
        print(i)
    obj = s3.get_object(Bucket='pronoo', Key=key)
    grid_sizes = pd.read_csv(obj['Body']).drop(['Unnamed: 0'], axis=1)
    all_players = pd.concat([all_players, grid_sizes], axis=0)
    
    
for i,key in enumerate(keysTournaments):
    if i%5000==0:
        print(i)
    obj = s3.get_object(Bucket='pronoo', Key=key)
    grid_sizes = pd.read_csv(obj['Body']).drop(['Unnamed: 0'], axis=1)
    all_tournaments = pd.concat([all_tournaments, grid_sizes], axis=0)

0
5000
10000
15000
20000
25000
30000
35000
40000
45000
50000
55000
60000
65000
70000


In [36]:
all_tournaments.to_csv('data/all_tournaments2020.csv', index=False)
all_players.to_csv('data/all_players2020.csv', index=False)    

In [30]:
all_players.to_csv('data/all_players2020.csv', index=False)    

In [37]:
pd.read_csv('data/all_tournaments.csv')

,name,phase,surfaceInout,typeTournament,date,winnerRate,looserRate,looserName,winnerName,winnerRateSurface,looserRateSurface,url
0,paris,Qualification,Indoor Carpet,1000,2000.11.13 - 2000.11.19,100% Wins,0% Wins,alex-obrien,albert-costa,1.000000,0.000000,https://www.atptour.com/en/players/fedex-head-...
1,paris,Qualification,Indoor Carpet,1000,2000.11.13 - 2000.11.19,100% Wins,0% Wins,gianluca-pozzi,albert-costa,1.000000,0.000000,https://www.atptour.com/en/players/fedex-head-...
2,paris,Qualification,Indoor Carpet,1000,2000.11.13 - 2000.11.19,30% Wins,70% Wins,tim-henman,albert-costa,1.000000,0.000000,https://www.atptour.com/en/players/fedex-head-...
3,paris,Qualification,Indoor Carpet,1000,2000.11.13 - 2000.11.19,50% Wins,50% Wins,wayne-ferreira,alex-corretja,1.000000,0.000000,https://www.atptour.com/en/players/fedex-head-...
4,paris,Qualification,Indoor Carpet,1000,2000.11.13 - 2000.11.19,100% Wins,0% Wins,jiri-novak,chris-woodruff,1.000000,0.000000,https://www.atptour.com/en/players/fedex-head-...
5,paris,Qualification,Indoor Carpet,1000,2000.11.13 - 2000.11.19,100% Wins,0% Wins,franco-squillari,david-prinosil,1.000000,0.000000,https://www.atptour.com/en/players/fedex-head-...
6,paris,Qualification,Indoor Carpet,1000,2000.11.13 - 2000.11.19,100% Wins,0% Wins,gaston-gaudio,david-prinosil,1.000000,0.000000,https://www.atptour.com/en/players/fedex-head-...
7,paris,Qualification,Indoor Carpet,1000,2000.11.13 - 2000.11.19,17% Wins,83% Wins,michael-chang,david-prinosil,1.000000,0.000000,https://www.atptour.com/en/players/fedex-head-...
8,paris,Qualification,Indoor Carpet,1000,2000.11.13 - 2000.11.19,67% Wins,33% Wins,roger-federer,dominik-hrbaty,1.000000,0.000000,https://www.atptour.com/en/players/fedex-head-...
9,paris,Qualification,Indoor Carpet,1000,2000.11.13 - 2000.11.19,75% Wins,25% Wins,carlos-moya,fabrice-santoro,1.000000,0.000000,https://www.atptour.com/en/players/fedex-head-...


In [ ]:
changer all tournement vers 2020 dans l'export, 
rajouter la condition 2020 dans la recherche de clés